# 第13章: 実験と準実験 (Experiments and Quasi-Experiments)

この章では、因果効果を推定するための実験的・準実験的手法について学びます。

## 主なトピック
- ランダム化比較試験 (RCT)
- 差の差分析 (Difference-in-Differences: DID)
- 回帰不連続デザイン (Regression Discontinuity Design: RDD)

## 必要なパッケージ

In [ ]:
# 必要なパッケージのインストールと読み込み
if (!require("AER")) install.packages("AER")
if (!require("dplyr")) install.packages("dplyr")
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("lmtest")) install.packages("lmtest")
if (!require("sandwich")) install.packages("sandwich")

library(AER)
library(dplyr)
library(ggplot2)
library(lmtest)
library(sandwich)

---
## 問題1: ランダム化比較試験の基礎

ある教育介入プログラムの効果を測定するために、100人の学生をランダムに処置群（50人）と対照群（50人）に分けました。

以下のシミュレーションデータを使って、処置効果を推定してください。

```r
set.seed(123)
n <- 100
treatment <- c(rep(1, 50), rep(0, 50))
# 真の処置効果は5点
y <- 70 + 5 * treatment + rnorm(n, 0, 10)
rct_data <- data.frame(treatment = treatment, score = y)
```

1. 処置群と対照群の平均点を計算してください
2. 単純な回帰分析で処置効果を推定してください
3. 推定された処置効果は真の値（5点）に近いですか？

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(123)
n <- 100
treatment <- c(rep(1, 50), rep(0, 50))
y <- 70 + 5 * treatment + rnorm(n, 0, 10)
rct_data <- data.frame(treatment = treatment, score = y)

# 1. 処置群と対照群の平均点
cat("処置群の平均点:", mean(rct_data$score[rct_data$treatment == 1]), "\n")
cat("対照群の平均点:", mean(rct_data$score[rct_data$treatment == 0]), "\n")
cat("平均の差:", mean(rct_data$score[rct_data$treatment == 1]) - mean(rct_data$score[rct_data$treatment == 0]), "\n\n")

# 2. 回帰分析による処置効果の推定
model_rct <- lm(score ~ treatment, data = rct_data)
summary(model_rct)

# 3. 推定値と真の値の比較
cat("\n推定された処置効果:", coef(model_rct)["treatment"], "\n")
cat("真の処置効果: 5\n")
cat("推定値は真の値に近い値を示しています。")

---
## 問題2: RCTにおける共変量の調整

問題1のデータに、学生の事前テストスコア（共変量）を追加します。

```r
set.seed(123)
n <- 100
treatment <- c(rep(1, 50), rep(0, 50))
pretest <- rnorm(n, 60, 15)
y <- 20 + 0.8 * pretest + 5 * treatment + rnorm(n, 0, 8)
rct_data2 <- data.frame(treatment = treatment, pretest = pretest, score = y)
```

1. 共変量なしで処置効果を推定してください
2. 共変量（pretest）を含めて処置効果を推定してください
3. 共変量を含めることで標準誤差がどう変化するか比較してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(123)
n <- 100
treatment <- c(rep(1, 50), rep(0, 50))
pretest <- rnorm(n, 60, 15)
y <- 20 + 0.8 * pretest + 5 * treatment + rnorm(n, 0, 8)
rct_data2 <- data.frame(treatment = treatment, pretest = pretest, score = y)

# 1. 共変量なしのモデル
model_simple <- lm(score ~ treatment, data = rct_data2)
cat("共変量なしのモデル:\n")
summary(model_simple)

# 2. 共変量ありのモデル
model_cov <- lm(score ~ treatment + pretest, data = rct_data2)
cat("\n共変量ありのモデル:\n")
summary(model_cov)

# 3. 標準誤差の比較
se_simple <- summary(model_simple)$coefficients["treatment", "Std. Error"]
se_cov <- summary(model_cov)$coefficients["treatment", "Std. Error"]

cat("\n標準誤差の比較:\n")
cat("共変量なし:", se_simple, "\n")
cat("共変量あり:", se_cov, "\n")
cat("標準誤差の減少率:", (se_simple - se_cov) / se_simple * 100, "%\n")
cat("\n共変量を含めることで残差の分散が減少し、推定の精度が向上します。")

---
## 問題3: 差の差分析（DID）の基礎

ある政策が2つの地域のうち1つでのみ実施されました。政策実施前後のデータがあります。

```r
set.seed(42)
did_data <- data.frame(
  region = rep(c("treatment", "control"), each = 100),
  period = rep(c("before", "after"), times = 100),
  outcome = c(
    rnorm(50, 50, 5),  # treatment, before
    rnorm(50, 60, 5),  # treatment, after (真の効果 = 5)
    rnorm(50, 45, 5),  # control, before
    rnorm(50, 50, 5)   # control, after
  )
)
```

1. 各グループ・期間の平均を計算してください
2. DID推定量を手計算で求めてください
3. 回帰分析でDID推定を行ってください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(42)
did_data <- data.frame(
  region = rep(c("treatment", "control"), each = 100),
  period = rep(c("before", "after"), times = 100),
  outcome = c(
    rnorm(50, 50, 5),  # treatment, before
    rnorm(50, 60, 5),  # treatment, after (真の効果 = 5)
    rnorm(50, 45, 5),  # control, before
    rnorm(50, 50, 5)   # control, after
  )
)

# ダミー変数の作成
did_data$treat <- ifelse(did_data$region == "treatment", 1, 0)
did_data$post <- ifelse(did_data$period == "after", 1, 0)

# 1. 各グループ・期間の平均
means <- aggregate(outcome ~ region + period, data = did_data, mean)
print(means)

# 2. DID推定量の手計算
mean_treat_after <- mean(did_data$outcome[did_data$treat == 1 & did_data$post == 1])
mean_treat_before <- mean(did_data$outcome[did_data$treat == 1 & did_data$post == 0])
mean_control_after <- mean(did_data$outcome[did_data$treat == 0 & did_data$post == 1])
mean_control_before <- mean(did_data$outcome[did_data$treat == 0 & did_data$post == 0])

did_estimate <- (mean_treat_after - mean_treat_before) - (mean_control_after - mean_control_before)
cat("\nDID推定量（手計算）:", did_estimate, "\n")

# 3. 回帰分析によるDID
model_did <- lm(outcome ~ treat + post + treat:post, data = did_data)
summary(model_did)

cat("\nDID推定量（回帰分析）:", coef(model_did)["treat:post"], "\n")
cat("真の処置効果: 5\n")

---
## 問題4: DIDの平行トレンド仮定

DIDの重要な仮定である「平行トレンド仮定」について考えます。

以下の2つのシナリオについて、DID推定が適切かどうか検討してください。

```r
# シナリオA: 平行トレンドが成り立つ
set.seed(1)
time <- 1:10
treatment_trend_A <- 30 + 2 * time + ifelse(time > 5, 10, 0) + rnorm(10, 0, 2)
control_trend_A <- 25 + 2 * time + rnorm(10, 0, 2)

# シナリオB: 平行トレンドが成り立たない
treatment_trend_B <- 30 + 3 * time + ifelse(time > 5, 10, 0) + rnorm(10, 0, 2)
control_trend_B <- 25 + 1 * time + rnorm(10, 0, 2)
```

1. 両シナリオのグラフを作成してください
2. どちらのシナリオでDIDが適切に使えるか説明してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# シナリオA: 平行トレンドが成り立つ
set.seed(1)
time <- 1:10
treatment_trend_A <- 30 + 2 * time + ifelse(time > 5, 10, 0) + rnorm(10, 0, 2)
control_trend_A <- 25 + 2 * time + rnorm(10, 0, 2)

# シナリオB: 平行トレンドが成り立たない
set.seed(1)
treatment_trend_B <- 30 + 3 * time + ifelse(time > 5, 10, 0) + rnorm(10, 0, 2)
control_trend_B <- 25 + 1 * time + rnorm(10, 0, 2)

# グラフの作成
par(mfrow = c(1, 2))

# シナリオA
plot(time, treatment_trend_A, type = "b", col = "blue", ylim = c(20, 70),
     main = "シナリオA: 平行トレンド", xlab = "時間", ylab = "アウトカム", pch = 16)
lines(time, control_trend_A, type = "b", col = "red", pch = 16)
abline(v = 5.5, lty = 2)
legend("topleft", legend = c("処置群", "対照群"), col = c("blue", "red"), lty = 1, pch = 16)

# シナリオB
plot(time, treatment_trend_B, type = "b", col = "blue", ylim = c(20, 70),
     main = "シナリオB: 非平行トレンド", xlab = "時間", ylab = "アウトカム", pch = 16)
lines(time, control_trend_B, type = "b", col = "red", pch = 16)
abline(v = 5.5, lty = 2)
legend("topleft", legend = c("処置群", "対照群"), col = c("blue", "red"), lty = 1, pch = 16)

par(mfrow = c(1, 1))

cat("\n解説:\n")
cat("シナリオA: 処置前のトレンドが平行（両群とも傾き2）なので、DIDが適切に使えます。\n")
cat("シナリオB: 処置前のトレンドが異なる（処置群は傾き3、対照群は傾き1）ため、\n")
cat("          DIDは処置効果を過大または過小に推定する可能性があります。\n")

---
## 問題5: 回帰不連続デザイン（RDD）の基礎

奨学金プログラムでは、試験スコア60点以上の学生が奨学金を受け取ります。

```r
set.seed(456)
n <- 200
test_score <- runif(n, 40, 80)
scholarship <- ifelse(test_score >= 60, 1, 0)
# 奨学金の効果は成績を10ポイント上昇させる
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
rdd_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)
```

1. データを可視化し、閾値での不連続性を確認してください
2. 単純なRDD回帰で効果を推定してください
3. 閾値を中心としたスコアを使って推定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(456)
n <- 200
test_score <- runif(n, 40, 80)
scholarship <- ifelse(test_score >= 60, 1, 0)
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
rdd_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)

# 1. データの可視化
plot(rdd_data$test_score, rdd_data$gpa, 
     col = ifelse(rdd_data$scholarship == 1, "blue", "red"),
     pch = 16, xlab = "試験スコア", ylab = "GPA",
     main = "RDD: 奨学金と成績の関係")
abline(v = 60, lty = 2, lwd = 2)
legend("topleft", legend = c("奨学金あり", "奨学金なし"), 
       col = c("blue", "red"), pch = 16)

# 2. 単純なRDD回帰
model_rdd_simple <- lm(gpa ~ test_score + scholarship, data = rdd_data)
cat("単純なRDD回帰:\n")
summary(model_rdd_simple)

# 3. 閾値を中心としたスコア（センタリング）
rdd_data$score_centered <- rdd_data$test_score - 60
model_rdd_centered <- lm(gpa ~ score_centered + scholarship, data = rdd_data)
cat("\nセンタリングしたRDD回帰:\n")
summary(model_rdd_centered)

cat("\n推定された奨学金効果:", coef(model_rdd_centered)["scholarship"], "\n")
cat("真の効果: 0.5\n")

---
## 問題6: RDDにおける異なる傾きの許容

問題5のデータを使って、閾値の前後で傾きが異なることを許容したRDDモデルを推定してください。

1. 閾値の前後で傾きが異なるモデルを推定してください
2. 傾きが同じモデルと比較してください
3. 交互作用項の係数は何を意味するか説明してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# 問題5のデータを再利用
set.seed(456)
n <- 200
test_score <- runif(n, 40, 80)
scholarship <- ifelse(test_score >= 60, 1, 0)
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
rdd_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)
rdd_data$score_centered <- rdd_data$test_score - 60

# 1. 傾きが異なるモデル
model_diff_slopes <- lm(gpa ~ score_centered + scholarship + score_centered:scholarship, 
                        data = rdd_data)
cat("傾きが異なることを許容したモデル:\n")
summary(model_diff_slopes)

# 2. 傾きが同じモデルとの比較
model_same_slopes <- lm(gpa ~ score_centered + scholarship, data = rdd_data)

cat("\nモデルの比較（F検定）:\n")
anova(model_same_slopes, model_diff_slopes)

# 3. 解説
cat("\n解説:\n")
cat("score_centered:scholarship の係数は、閾値の上下でスコアとGPAの関係（傾き）が\n")
cat("どれだけ異なるかを表します。\n")
cat("\nこの係数が統計的に有意でない場合、傾きは同じと仮定してもよいでしょう。\n")

# グラフで確認
plot(rdd_data$test_score, rdd_data$gpa, 
     col = ifelse(rdd_data$scholarship == 1, "blue", "red"),
     pch = 16, xlab = "試験スコア", ylab = "GPA",
     main = "RDD: 異なる傾きを許容")
abline(v = 60, lty = 2, lwd = 2)

# 回帰直線を追加
abline(a = coef(model_diff_slopes)[1] - 60 * coef(model_diff_slopes)[2], 
       b = coef(model_diff_slopes)[2], col = "red", lwd = 2)
abline(a = coef(model_diff_slopes)[1] + coef(model_diff_slopes)[3] - 
       60 * (coef(model_diff_slopes)[2] + coef(model_diff_slopes)[4]), 
       b = coef(model_diff_slopes)[2] + coef(model_diff_slopes)[4], col = "blue", lwd = 2)

---
## 問題7: RDDのバンド幅選択

RDDでは閾値周辺のデータのみを使用する「局所線形回帰」が一般的です。

問題5のデータを使って、異なるバンド幅での推定結果を比較してください。

1. 閾値から±5点以内のデータで推定してください
2. 閾値から±10点以内のデータで推定してください
3. 閾値から±20点以内のデータで推定してください
4. バンド幅の選択がどのように結果に影響するか考察してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# 問題5のデータを再利用
set.seed(456)
n <- 200
test_score <- runif(n, 40, 80)
scholarship <- ifelse(test_score >= 60, 1, 0)
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
rdd_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)
rdd_data$score_centered <- rdd_data$test_score - 60

# 異なるバンド幅での推定
bandwidths <- c(5, 10, 20)
results <- data.frame(bandwidth = numeric(), estimate = numeric(), 
                      se = numeric(), n_obs = numeric())

for (bw in bandwidths) {
  subset_data <- rdd_data[abs(rdd_data$score_centered) <= bw, ]
  model <- lm(gpa ~ score_centered + scholarship, data = subset_data)
  
  results <- rbind(results, data.frame(
    bandwidth = bw,
    estimate = coef(model)["scholarship"],
    se = summary(model)$coefficients["scholarship", "Std. Error"],
    n_obs = nrow(subset_data)
  ))
}

cat("異なるバンド幅での推定結果:\n")
print(results)

# 可視化
par(mfrow = c(1, 3))
for (bw in bandwidths) {
  subset_data <- rdd_data[abs(rdd_data$score_centered) <= bw, ]
  plot(subset_data$test_score, subset_data$gpa, 
       col = ifelse(subset_data$scholarship == 1, "blue", "red"),
       pch = 16, main = paste("バンド幅 ±", bw, "点"),
       xlab = "試験スコア", ylab = "GPA")
  abline(v = 60, lty = 2)
}
par(mfrow = c(1, 1))

cat("\n考察:\n")
cat("- 狭いバンド幅: バイアスは小さいが、サンプルサイズが減り分散が大きくなる\n")
cat("- 広いバンド幅: サンプルサイズは大きいが、閾値から遠いデータを含むためバイアスが生じうる\n")
cat("- 最適なバンド幅はバイアスと分散のトレードオフで決定される\n")

---
## 問題8: ファジーRDD

シャープRDDでは閾値で処置が完全に決定されますが、ファジーRDDでは処置確率が変化するだけです。

```r
set.seed(789)
n <- 300
test_score <- runif(n, 40, 80)
# 閾値を超えると奨学金を受ける確率が上昇（完全ではない）
prob_scholarship <- ifelse(test_score >= 60, 0.8, 0.2)
scholarship <- rbinom(n, 1, prob_scholarship)
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
fuzzy_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)
```

1. 閾値の前後での奨学金受給率を確認してください
2. 通常のOLS回帰で効果を推定してください
3. 閾値を操作変数として2SLS推定を行ってください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(789)
n <- 300
test_score <- runif(n, 40, 80)
prob_scholarship <- ifelse(test_score >= 60, 0.8, 0.2)
scholarship <- rbinom(n, 1, prob_scholarship)
gpa <- 2.0 + 0.03 * test_score + 0.5 * scholarship + rnorm(n, 0, 0.3)
fuzzy_data <- data.frame(test_score = test_score, scholarship = scholarship, gpa = gpa)
fuzzy_data$above_cutoff <- ifelse(fuzzy_data$test_score >= 60, 1, 0)
fuzzy_data$score_centered <- fuzzy_data$test_score - 60

# 1. 閾値前後での奨学金受給率
cat("閾値未満（スコア < 60）での奨学金受給率:", 
    mean(fuzzy_data$scholarship[fuzzy_data$above_cutoff == 0]), "\n")
cat("閾値以上（スコア >= 60）での奨学金受給率:", 
    mean(fuzzy_data$scholarship[fuzzy_data$above_cutoff == 1]), "\n\n")

# 2. 通常のOLS回帰
model_ols <- lm(gpa ~ score_centered + scholarship, data = fuzzy_data)
cat("OLS推定:\n")
summary(model_ols)

# 3. 2SLS推定（閾値を操作変数として使用）
model_2sls <- ivreg(gpa ~ score_centered + scholarship | score_centered + above_cutoff, 
                    data = fuzzy_data)
cat("\n2SLS推定（ファジーRDD）:\n")
summary(model_2sls)

cat("\n比較:\n")
cat("OLS推定値:", coef(model_ols)["scholarship"], "\n")
cat("2SLS推定値:", coef(model_2sls)["scholarship"], "\n")
cat("真の効果: 0.5\n")
cat("\nファジーRDDでは、閾値を超えることによる処置確率の変化を利用して\n")
cat("局所的な平均処置効果（LATE）を推定します。\n")

---
## 問題9: DIDにおける複数期間の分析

複数期間のパネルデータを使ったDID分析を行います。

```r
set.seed(111)
n_units <- 50
n_periods <- 6
treatment_time <- 4  # 期間4から処置開始

panel_did <- expand.grid(unit = 1:n_units, time = 1:n_periods)
panel_did$treat_group <- ifelse(panel_did$unit <= 25, 1, 0)
panel_did$post <- ifelse(panel_did$time >= treatment_time, 1, 0)
panel_did$treated <- panel_did$treat_group * panel_did$post

# 固定効果とトレンドを含むアウトカム
unit_fe <- rnorm(n_units, 0, 5)
time_fe <- c(0, 2, 4, 6, 8, 10)
panel_did$y <- 50 + unit_fe[panel_did$unit] + time_fe[panel_did$time] + 
               10 * panel_did$treated + rnorm(n_units * n_periods, 0, 3)
```

1. グループ別の時系列推移を可視化してください
2. 固定効果を含むDIDモデルを推定してください
3. クラスター頑健標準誤差を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(111)
n_units <- 50
n_periods <- 6
treatment_time <- 4

panel_did <- expand.grid(unit = 1:n_units, time = 1:n_periods)
panel_did$treat_group <- ifelse(panel_did$unit <= 25, 1, 0)
panel_did$post <- ifelse(panel_did$time >= treatment_time, 1, 0)
panel_did$treated <- panel_did$treat_group * panel_did$post

unit_fe <- rnorm(n_units, 0, 5)
time_fe <- c(0, 2, 4, 6, 8, 10)
panel_did$y <- 50 + unit_fe[panel_did$unit] + time_fe[panel_did$time] + 
               10 * panel_did$treated + rnorm(n_units * n_periods, 0, 3)

# 1. グループ別の時系列推移の可視化
group_means <- aggregate(y ~ time + treat_group, data = panel_did, mean)

plot(1:n_periods, group_means$y[group_means$treat_group == 1], 
     type = "b", col = "blue", pch = 16, ylim = c(45, 75),
     xlab = "期間", ylab = "アウトカム", main = "グループ別の時系列推移")
lines(1:n_periods, group_means$y[group_means$treat_group == 0], 
      type = "b", col = "red", pch = 16)
abline(v = treatment_time - 0.5, lty = 2, lwd = 2)
legend("topleft", legend = c("処置群", "対照群"), col = c("blue", "red"), 
       lty = 1, pch = 16)

# 2. 固定効果を含むDIDモデル
panel_did$unit <- factor(panel_did$unit)
panel_did$time <- factor(panel_did$time)

model_twfe <- lm(y ~ treated + unit + time, data = panel_did)
cat("Two-way Fixed Effects DIDモデル:\n")
cat("処置効果の推定値:", coef(model_twfe)["treated"], "\n")
cat("（ユニット固定効果と時間固定効果は省略）\n\n")

# 3. クラスター頑健標準誤差
cluster_se <- coeftest(model_twfe, vcov = vcovCL(model_twfe, cluster = panel_did$unit))
cat("クラスター頑健標準誤差を使用した結果:\n")
print(cluster_se["treated", ])

cat("\n真の処置効果: 10\n")

---
## 問題10: 処置効果の異質性

RCTのデータを使って、処置効果が共変量によって異なるかを検討します。

```r
set.seed(222)
n <- 400
treatment <- rbinom(n, 1, 0.5)
gender <- rbinom(n, 1, 0.5)  # 0 = 女性, 1 = 男性
age <- sample(18:65, n, replace = TRUE)

# 処置効果は性別によって異なる（女性: 5, 男性: 10）
treatment_effect <- 5 + 5 * gender
y <- 50 + treatment_effect * treatment + 0.1 * age + rnorm(n, 0, 10)
heterog_data <- data.frame(treatment = treatment, gender = gender, age = age, y = y)
```

1. 全体の処置効果を推定してください
2. 性別ごとの処置効果を推定してください
3. 交互作用項を使って処置効果の異質性を検定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(222)
n <- 400
treatment <- rbinom(n, 1, 0.5)
gender <- rbinom(n, 1, 0.5)
age <- sample(18:65, n, replace = TRUE)

treatment_effect <- 5 + 5 * gender
y <- 50 + treatment_effect * treatment + 0.1 * age + rnorm(n, 0, 10)
heterog_data <- data.frame(treatment = treatment, gender = gender, age = age, y = y)

# 1. 全体の処置効果
model_overall <- lm(y ~ treatment + age, data = heterog_data)
cat("全体の処置効果:\n")
summary(model_overall)

# 2. 性別ごとの処置効果
model_female <- lm(y ~ treatment + age, data = heterog_data[heterog_data$gender == 0, ])
model_male <- lm(y ~ treatment + age, data = heterog_data[heterog_data$gender == 1, ])

cat("\n女性の処置効果:", coef(model_female)["treatment"], "\n")
cat("男性の処置効果:", coef(model_male)["treatment"], "\n")

# 3. 交互作用項を使った検定
model_interaction <- lm(y ~ treatment * gender + age, data = heterog_data)
cat("\n交互作用モデル:\n")
summary(model_interaction)

cat("\n解釈:\n")
cat("- treatment: 女性（gender=0）における処置効果\n")
cat("- treatment:gender: 男性の処置効果と女性の処置効果の差\n")
cat("\n真の値:\n")
cat("女性の処置効果: 5\n")
cat("男性の処置効果: 10\n")
cat("処置効果の差: 5\n")

---
## まとめ

この章では以下の内容を学びました：

1. **ランダム化比較試験（RCT）**: 因果効果を推定するためのゴールドスタンダード
2. **共変量調整**: RCTでも共変量を含めることで推定精度が向上
3. **差の差分析（DID）**: 処置群と対照群の変化の差から因果効果を推定
4. **平行トレンド仮定**: DIDの妥当性を支える重要な仮定
5. **回帰不連続デザイン（RDD）**: 閾値周辺での不連続性を利用した因果推論
6. **バンド幅選択**: RDDにおけるバイアスと分散のトレードオフ
7. **ファジーRDD**: 処置確率が不連続に変化する場合の対処法
8. **処置効果の異質性**: サブグループごとに異なる処置効果の推定